# Generate Uganda Flood Timeseries (2020–2025)

This notebook computes a **flood level metric** (fraction of each district flooded) for Uganda from **2020–2025**, using:

- **Sentinel-1 SAR** flood detection
- **JRC Global Surface Water** (permanent water mask)
- **GPM IMERG** precipitation (optional)
- **Uganda district boundaries**

Outputs: **CSV dataset** with flood_fraction per district per month.

**Before running:** place your district shapefile as `uganda_admin2.shp` in the working directory (or update the path in the notebook).

In [ ]:
!pip install earthengine-api geemap geopandas pandas shapely rasterio --quiet

In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

print("Earth Engine initialized.")

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping

DISTRICT_SHP = "uganda_admin2.shp"
districts_gdf = gpd.read_file(DISTRICT_SHP)
districts_gdf = districts_gdf.to_crs(epsg=4326)

districts = []
for idx, row in districts_gdf.iterrows():
    name = row.get("NAME_2") or row.get("district") or f"district_{idx}"
    districts.append({
        "id": idx,
        "name": name,
        "geometry": mapping(row.geometry),
        "area_m2": row.geometry.area
    })

print(f"Loaded {len(districts)} districts.")

In [ ]:
import ee
S1 = ee.ImageCollection('COPERNICUS/S1_GRD')
GSW = ee.Image('JRC/GSW1_3/GlobalSurfaceWater')
GPM = ee.ImageCollection('NASA/GPM_L3/IMERG_V06')

print("Datasets loaded.")

In [ ]:
START = '2020-01-01'
END   = '2025-08-31'

BASE_START = '2017-01-01'
BASE_END   = '2019-12-31'

ANOMALY_THRESHOLD = -3.0  # dB

OUTPUT_CSV = "uganda_flood_timeseries_2020_2025.csv" 

In [ ]:
import geemap

uganda_ee = geemap.geopandas_to_ee(districts_gdf.dissolve())

def s1_vv(col, region):
    return (col
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filterBounds(region)
            .select('VV'))

baseline_by_month = {}
for m in range(1, 13):
    monthly = s1_vv(
        S1.filterDate(BASE_START, BASE_END)
          .filter(ee.Filter.calendarRange(m, m, 'month')),
        uganda_ee
    )
    baseline_by_month[m] = monthly.median()

print("Baseline images computed.")

In [ ]:
def detect_flood(start, end):
    start_dt = pd.to_datetime(start)
    month = int(start_dt.month)

    period = s1_vv(S1.filterDate(start, end), uganda_ee).median()
    baseline = baseline_by_month[month]
    anomaly = period.subtract(baseline)

    perm_water = GSW.select('occurrence').gte(50)
    flood = anomaly.lte(ANOMALY_THRESHOLD).And(perm_water.Not())

    return flood.rename('flood')

In [ ]:
def monthly_periods(start, end):
    rng = pd.date_range(start, end, freq='MS')
    periods = []
    for d in rng:
        periods.append((d.strftime('%Y-%m-%d'), (d + pd.offsets.MonthEnd(1)).strftime('%Y-%m-%d')))
    return periods

periods = monthly_periods(START, END)
periods[:3], periods[-3:]

In [ ]:
rows = []

for start, end in periods:
    print("Processing:", start, end)
    flood_img = detect_flood(start, end)
    area_img = ee.Image.pixelArea().updateMask(flood_img)

    for d in districts:
        geom = ee.Geometry(d['geometry'])
        stats = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geom,
            scale=10,
            maxPixels=1e13
        )
        flooded = 0
        try:
            flooded = stats.getInfo().get('area') if stats.getInfo() else 0
        except Exception:
            flooded = 0

        flood_fraction = flooded / d['area_m2'] if d['area_m2'] > 0 else None

        rows.append({
            'period_start': start,
            'period_end': end,
            'district_id': d['id'],
            'district_name': d['name'],
            'flooded_m2': flooded,
            'district_area_m2': d['area_m2'],
            'flood_fraction': flood_fraction
        })

In [ ]:
import pandas as pd
df = pd.DataFrame(rows)
df.to_csv(OUTPUT_CSV, index=False)
df.head()